In [13]:
from pathlib import Path
import pandas as pd
import numpy as np

def count_csv_rows(file_path, has_header=True):
    """
    Counts the number of rows in a CSV file.
    
    Parameters:
    - file_path (Path): The path to the CSV file.
    - has_header (bool): Whether the CSV file has a header row.
    
    Returns:
    - int: The number of data rows in the CSV file.
    """
    with file_path.open('r', encoding='utf-8') as file:
        row_count = sum(1 for _ in file)
    return row_count - 1 if has_header else row_count

def enumerate_and_sort_csv(datadir, has_header=True):
    """
    Enumerates CSV files in the given directory, counts their rows, and sorts them.
    
    Parameters:
    - datadir (str or Path): The directory containing CSV files.
    - has_header (bool): Whether the CSV files have header rows.
    
    Returns:
    - List of tuples: Each tuple contains (file_name, row_count), sorted by row_count ascending.
    """
    datadir = Path(datadir)
    if not datadir.is_dir():
        raise ValueError(f"The path {datadir} is not a valid directory.")
    
    # Find all CSV files in the directory
    csv_files = list(datadir.glob('*.csv'))
    
    if not csv_files:
        print("No CSV files found in the specified directory.")
        return []
    
    file_row_counts = []
    
    for file in csv_files:
        try:
            rows = count_csv_rows(file, has_header=has_header)
            file_row_counts.append((file.name, rows))
        except Exception as e:
            print(f"Error processing {file.name}: {e}")
    
    # Sort the list of tuples based on row_count (ascending order)
    sorted_files = sorted(file_row_counts, key=lambda x: x[1])
    
    return sorted_files




In [14]:
# Specify the directory containing CSV files
data_directory = 'data/prices'  # Replace with your directory path

# Enumerate and sort CSV files
sorted_csv_files = enumerate_and_sort_csv(data_directory, has_header=True)

if sorted_csv_files:
    print("\nCSV files sorted by number of rows (ascending):")
    for filename, count in sorted_csv_files:
        print(f"{filename}: {count} rows")


CSV files sorted by number of rows (ascending):
GEV.csv: 448 rows
SOLV.csv: 449 rows
VLTO.csv: 568 rows
KVUE.csv: 673 rows
GEHC.csv: 768 rows
CEG.csv: 997 rows
ABNB.csv: 1275 rows
CARR.csv: 1460 rows
OTIS.csv: 1460 rows
CTVA.csv: 1666 rows
UBER.csv: 1676 rows
DOW.csv: 1712 rows
FOX.csv: 1717 rows
FOXA.csv: 1718 rows
MRNA.csv: 1781 rows
DAY.csv: 1937 rows
VICI.csv: 2016 rows
IR.csv: 2177 rows
INVH.csv: 2247 rows
LW.csv: 2302 rows
HWM.csv: 2309 rows
FTV.csv: 2393 rows
HPE.csv: 2571 rows
KHC.csv: 2645 rows
PYPL.csv: 2645 rows
ETSY.csv: 2700 rows
QRVO.csv: 2771 rows
KEYS.csv: 2822 rows
CFG.csv: 2840 rows
CZR.csv: 2842 rows
SYF.csv: 2878 rows
ANET.csv: 2916 rows
PAYC.csv: 2952 rows
HLT.csv: 3036 rows
ALLE.csv: 3053 rows
CDW.csv: 3153 rows
NWS.csv: 3159 rows
NWSA.csv: 3159 rows
IQV.csv: 3187 rows
ZTS.csv: 3254 rows
NCLH.csv: 3263 rows
ABBV.csv: 3275 rows
FANG.csv: 3328 rows
PANW.csv: 3387 rows
NOW.csv: 3401 rows
META.csv: 3430 rows
AMCR.csv: 3433 rows
PSX.csv: 3456 rows
ENPH.csv: 3464 rows


In [8]:
from collections import defaultdict
import numpy as np

def cluster_csv_pairs_hdbscan_sorted(sorted_csv_files,
                                     min_cluster_size=5,
                                     min_samples=None,
                                     alpha=1.0):
    """
    HDBSCAN clustering with cluster IDs reordered by descending row counts.
    Cluster 0 has the largest rows.
    """

    import hdbscan
    from sklearn.preprocessing import RobustScaler

    if not sorted_csv_files:
        return {}

    # Feature: row count only
    row_counts = np.array([rc for _, rc in sorted_csv_files], dtype=np.float64).reshape(-1, 1)

    # Robust scaling
    X = RobustScaler().fit_transform(row_counts)

    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        alpha=alpha,
        metric="euclidean"
    )

    raw_labels = clusterer.fit_predict(X)

    # Collect raw clusters
    raw_clusters = defaultdict(list)
    for pair, lab in zip(sorted_csv_files, raw_labels):
        raw_clusters[int(lab)].append(pair)

    # Separate noise
    noise = raw_clusters.pop(-1, [])

    # Sort clusters by mean row count (descending)
    sorted_clusters = sorted(
        raw_clusters.values(),
        key=lambda items: np.mean([rc for _, rc in items]),
        reverse=True
    )

    # Reassign cluster IDs: 0 = largest rows
    final_clusters = {}
    for new_id, items in enumerate(sorted_clusters):
        final_clusters[new_id] = sorted(items, key=lambda x: x[1], reverse=True)

    if noise:
        final_clusters[-1] = sorted(noise, key=lambda x: x[1], reverse=True)

    # Pretty print
    print(f"Clusters found (excluding noise): {len(sorted_clusters)}")
    print(f"Noise points (-1): {len(noise)}\n")

    for cid in sorted(final_clusters.keys(), key=lambda x: (x == -1, x)):
        title = "Noise (-1)" if cid == -1 else f"Cluster {cid}"
        print(f"{title} | size={len(final_clusters[cid])}")
        for fname, rc in final_clusters[cid]:
            print(f"  ({fname!r}, {rc})")
        print()

    return final_clusters


# ---- Example usage ----
# clusters = cluster_csv_pairs_hdbscan_sorted(
#     sorted_csv_files,
#     min_cluster_size=5
# )


In [15]:
from collections import defaultdict
import numpy as np

def cluster_csv_pairs_k_mean_sorted(sorted_csv_files,
                                    n_clusters,
                                    random_state=None):
    """
    K-Means clustering with cluster IDs reordered by descending row counts.
    Cluster 0 has the largest rows, then 1, etc.

    Parameters
    ----------
    sorted_csv_files : List[Tuple[str, int]]
        A list of (filename, row_count) pairs.
    n_clusters : int
        The number of clusters to form.
    random_state : int or None
        Random seed for KMeans initialization.

    Returns
    -------
    dict[int, List[Tuple[str, int]]]
        A dict mapping new_cluster_id -> list of (filename, row_count),
        with filenames sorted by row count descending.
    """
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import RobustScaler

    if not sorted_csv_files:
        return {}

    # Extract row counts and reshape for sklearn
    row_counts = np.array([rc for _, rc in sorted_csv_files], dtype=np.float64).reshape(-1, 1)

    # Robust scaling to reduce influence of outliers
    X = RobustScaler().fit_transform(row_counts)

    # Run KMeans
    kmeans = KMeans(
        n_clusters=n_clusters,
        random_state=random_state
    )
    labels = kmeans.fit_predict(X)

    # Collect raw clusters
    raw_clusters = defaultdict(list)
    for (fname, rc), lab in zip(sorted_csv_files, labels):
        raw_clusters[int(lab)].append((fname, rc))

    # Re‐order clusters by descending mean row count
    sorted_cluster_items = sorted(
        raw_clusters.values(),
        key=lambda items: np.mean([rc for _, rc in items]),
        reverse=True
    )

    # Assign new IDs: 0 = largest mean row count, 1 = next largest, ...
    final_clusters = {}
    for new_id, items in enumerate(sorted_cluster_items):
        # Within each cluster sort by row count descending
        final_clusters[new_id] = sorted(items, key=lambda x: x[1], reverse=True)

    # Pretty print
    print(f"KMeans Clusters found: {len(final_clusters)}\n")
    for cid in sorted(final_clusters.keys()):
        print(f"Cluster {cid} | size={len(final_clusters[cid])}")
        for fname, rc in final_clusters[cid]:
            print(f"  ({fname!r}, {rc})")
        print()

    return final_clusters


# ---- Example usage ----
# Suppose you have:
# sorted_csv_files = [
#     ("a.csv", 150),
#     ("b.csv", 2000),
#     ("c.csv", 75),
#     # ...
# ]
# clusters = cluster_csv_pairs_k_mean_sorted(
#     sorted_csv_files,
#     n_clusters=4,
#     random_state=42
# )

In [16]:
clusters = cluster_csv_pairs_k_mean_sorted(sorted_csv_files,
                                              n_clusters=10,
                                              random_state=42)

KMeans Clusters found: 10

Cluster 0 | size=26
  ('AEP.csv', 16113)
  ('BA.csv', 16113)
  ('CAT.csv', 16113)
  ('CNP.csv', 16113)
  ('CVX.csv', 16113)
  ('DIS.csv', 16113)
  ('DTE.csv', 16113)
  ('ED.csv', 16113)
  ('GD.csv', 16113)
  ('GE.csv', 16113)
  ('HON.csv', 16113)
  ('HPQ.csv', 16113)
  ('IBM.csv', 16113)
  ('IP.csv', 16113)
  ('JNJ.csv', 16113)
  ('KO.csv', 16113)
  ('KR.csv', 16113)
  ('MMM.csv', 16113)
  ('MO.csv', 16113)
  ('MRK.csv', 16113)
  ('MSI.csv', 16113)
  ('PG.csv', 16113)
  ('XOM.csv', 16113)
  ('LMT.csv', 16112)
  ('RTX.csv', 16050)
  ('MCD.csv', 14978)

Cluster 1 | size=63
  ('AXP.csv', 13514)
  ('BMY.csv', 13514)
  ('DD.csv', 13514)
  ('DE.csv', 13514)
  ('EMR.csv', 13514)
  ('ETN.csv', 13514)
  ('ETR.csv', 13514)
  ('F.csv', 13514)
  ('HAL.csv', 13514)
  ('LLY.csv', 13514)
  ('PCG.csv', 13514)
  ('PEP.csv', 13514)
  ('PFE.csv', 13514)
  ('TXN.csv', 13514)
  ('WFC.csv', 13514)
  ('WMT.csv', 13454)
  ('HUBB.csv', 13392)
  ('AIG.csv', 13368)
  ('IPG.csv', 13358)

In [ ]:
output_dir = "input_files"
# create if it does not exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

for cluster_id in range(len(clusters)-1):
    print(f"Processing cluster {cluster_id}")
    dataframes_vol = []
    for filename, rows in clusters[cluster_id]:
        print(f"File: {filename}, Rows: {rows}")
        # make one single pandas dataframe contains the data from above stock IDs, the index is the data, the columns are the stock IDs
        stock_id = filename[:-4]  

        file_path = Path(data_directory) / filename
        df_vol = pd.read_csv(file_path, usecols=['Date', 'Adj Close', 'Volume'], parse_dates=['Date'])
        df_vol.set_index('Date', inplace=True)
        df_vol.rename(columns={'Adj Close': f"{stock_id}_price", 'Volume': f"{stock_id}_vol"}, inplace=True)
        dataframes_vol.append(df_vol)

    combined_df_vol = pd.concat(dataframes_vol, axis=1)
    combined_df_vol.fillna(method='ffill', inplace=True)
    combined_df_vol.fillna(method='bfill', inplace=True)
    log_returns_df_vol = np.log(combined_df_vol / (combined_df_vol.shift(1) + 1e-9 ) + 1e-9)
    # drop the first row
    #log_returns_df_vol = log_returns_df_vol.iloc[1:]
    log_returns_df_vol = log_returns_df_vol.dropna()  # Drop the first row with

    price_columns = [col for col in log_returns_df_vol.columns if col.endswith('_price')]
    volume_columns = [col for col in log_returns_df_vol.columns if col.endswith('_vol')]
    log_returns_df_vol = log_returns_df_vol[price_columns + volume_columns]
    # find out the earliest date in the dataframe
    start_date = log_returns_df_vol.index.min().strftime('%Y-%m-%d')
    log_returns_df_vol.to_csv(f"{output_dir}/stock_data_vol-cluster-{cluster_id}-startdate-{start_date}.csv")

Processing cluster 0
File: AEP.csv, Rows: 16113
File: BA.csv, Rows: 16113
File: CAT.csv, Rows: 16113
File: CNP.csv, Rows: 16113
File: CVX.csv, Rows: 16113
File: DIS.csv, Rows: 16113
File: DTE.csv, Rows: 16113
File: ED.csv, Rows: 16113
File: GD.csv, Rows: 16113
File: GE.csv, Rows: 16113
File: HON.csv, Rows: 16113
File: HPQ.csv, Rows: 16113
File: IBM.csv, Rows: 16113
File: IP.csv, Rows: 16113
File: JNJ.csv, Rows: 16113
File: KO.csv, Rows: 16113
File: KR.csv, Rows: 16113
File: MMM.csv, Rows: 16113
File: MO.csv, Rows: 16113
File: MRK.csv, Rows: 16113
File: MSI.csv, Rows: 16113
File: PG.csv, Rows: 16113
File: XOM.csv, Rows: 16113
File: LMT.csv, Rows: 16112
File: RTX.csv, Rows: 16050
File: MCD.csv, Rows: 14978


/tmp/ipykernel_6643/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_6643/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 1
File: AXP.csv, Rows: 13514
File: BMY.csv, Rows: 13514
File: DD.csv, Rows: 13514
File: DE.csv, Rows: 13514
File: EMR.csv, Rows: 13514
File: ETN.csv, Rows: 13514
File: ETR.csv, Rows: 13514
File: F.csv, Rows: 13514
File: HAL.csv, Rows: 13514
File: LLY.csv, Rows: 13514
File: PCG.csv, Rows: 13514
File: PEP.csv, Rows: 13514
File: PFE.csv, Rows: 13514
File: TXN.csv, Rows: 13514
File: WFC.csv, Rows: 13514
File: WMT.csv, Rows: 13454
File: HUBB.csv, Rows: 13392
File: AIG.csv, Rows: 13368
File: IPG.csv, Rows: 13358
File: K.csv, Rows: 13336
File: AVY.csv, Rows: 13334
File: BAC.csv, Rows: 13334
File: BALL.csv, Rows: 13334
File: BDX.csv, Rows: 13334
File: CLX.csv, Rows: 13334
File: CMI.csv, Rows: 13334
File: CMS.csv, Rows: 13334
File: CPB.csv, Rows: 13334
File: CVS.csv, Rows: 13334
File: ECL.csv, Rows: 13334
File: ES.csv, Rows: 13334
File: EVRG.csv, Rows: 13334
File: GWW.csv, Rows: 13334
File: LNT.csv, Rows: 13334
File: MKC.csv, Rows: 13334
File: MMC.csv, Rows: 13334
File: NEE.c

/tmp/ipykernel_6643/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_6643/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 2
File: C.csv, Rows: 12357
File: FDX.csv, Rows: 12036
File: WDC.csv, Rows: 11895
File: DHR.csv, Rows: 11854
File: APA.csv, Rows: 11760
File: LUV.csv, Rows: 11600
File: PEG.csv, Rows: 11600
File: UNP.csv, Rows: 11600
File: BIO.csv, Rows: 11561
File: ABT.csv, Rows: 11548
File: ADI.csv, Rows: 11548
File: ADM.csv, Rows: 11548
File: ADP.csv, Rows: 11548
File: AFL.csv, Rows: 11548
File: AMAT.csv, Rows: 11548
File: AMD.csv, Rows: 11548
File: APD.csv, Rows: 11548
File: CAG.csv, Rows: 11548
File: CHD.csv, Rows: 11548
File: CINF.csv, Rows: 11548
File: CMA.csv, Rows: 11548
File: CMCSA.csv, Rows: 11548
File: D.csv, Rows: 11548
File: DOV.csv, Rows: 11548
File: DUK.csv, Rows: 11548
File: EFX.csv, Rows: 11548
File: EQT.csv, Rows: 11548
File: FITB.csv, Rows: 11548
File: FMC.csv, Rows: 11548
File: GIS.csv, Rows: 11548
File: GPC.csv, Rows: 11548
File: HAS.csv, Rows: 11548
File: HBAN.csv, Rows: 11548
File: HRL.csv, Rows: 11548
File: HST.csv, Rows: 11548
File: HSY.csv, Rows: 11548
File:

...

/tmp/ipykernel_6643/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_6643/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 7
File: MOH.csv, Rows: 5667
File: LKQ.csv, Rows: 5601
File: NRG.csv, Rows: 5561
File: AIZ.csv, Rows: 5517
File: CBRE.csv, Rows: 5430
File: CRM.csv, Rows: 5422
File: DPZ.csv, Rows: 5409
File: EXR.csv, Rows: 5385
File: GOOG.csv, Rows: 5382
File: GOOGL.csv, Rows: 5382
File: DLR.csv, Rows: 5332
File: MKTX.csv, Rows: 5327
File: MPWR.csv, Rows: 5317
File: LVS.csv, Rows: 5300
File: CE.csv, Rows: 5275
File: DXCM.csv, Rows: 5218
File: BLDR.csv, Rows: 5166
File: WBD.csv, Rows: 5159
File: EXPE.csv, Rows: 5150
File: CF.csv, Rows: 5135
File: AMP.csv, Rows: 5111
File: AAL.csv, Rows: 5103
File: ICE.csv, Rows: 5067
File: LYV.csv, Rows: 5043
File: CMG.csv, Rows: 5020
File: UAL.csv, Rows: 5013
File: TDG.csv, Rows: 4987
File: MA.csv, Rows: 4937
File: LDOS.csv, Rows: 4837
File: FSLR.csv, Rows: 4814
File: BR.csv, Rows: 4731
File: SMCI.csv, Rows: 4726
File: TMUS.csv, Rows: 4712
File: DAL.csv, Rows: 4702
File: PODD.csv, Rows: 4694
File: TEL.csv, Rows: 4673
File: BX.csv, Rows: 4667
File: LU

/tmp/ipykernel_6643/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_6643/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 8
File: AVGO.csv, Rows: 4132
File: VRSK.csv, Rows: 4089
File: DG.csv, Rows: 4062
File: FTNT.csv, Rows: 4059
File: CHTR.csv, Rows: 4028
File: GNRC.csv, Rows: 4002
File: LYB.csv, Rows: 3950
File: CBOE.csv, Rows: 3917
File: TSLA.csv, Rows: 3907
File: NXPI.csv, Rows: 3880
File: GM.csv, Rows: 3807
File: TRGP.csv, Rows: 3795
File: CPAY.csv, Rows: 3789
File: KMI.csv, Rows: 3749
File: HCA.csv, Rows: 3731
File: HII.csv, Rows: 3723
File: MPC.csv, Rows: 3657
File: XYL.csv, Rows: 3580
File: APTV.csv, Rows: 3555
File: EPAM.csv, Rows: 3500
File: ENPH.csv, Rows: 3464
File: PSX.csv, Rows: 3456
File: AMCR.csv, Rows: 3433
File: META.csv, Rows: 3430
File: NOW.csv, Rows: 3401
File: PANW.csv, Rows: 3387
File: FANG.csv, Rows: 3328
File: ABBV.csv, Rows: 3275
File: NCLH.csv, Rows: 3263
File: ZTS.csv, Rows: 3254
File: IQV.csv, Rows: 3187
File: NWS.csv, Rows: 3159
File: NWSA.csv, Rows: 3159
File: CDW.csv, Rows: 3153
File: ALLE.csv, Rows: 3053
File: HLT.csv, Rows: 3036
File: PAYC.csv, Rows: 29

/tmp/ipykernel_6643/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_6643/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


In [18]:
from pathlib import Path
import pandas as pd

# to get a big table for all stocks
dataframes_vol = []

for cluster_id in range(len(clusters) - 1):

    # sort files in THIS cluster by row count (descending) BEFORE reading/appending
    for filename, rows in sorted(clusters[cluster_id], key=lambda x: x[1], reverse=True):
        print(f"File: {filename}, Rows: {rows}")

        stock_id = filename[:-4]
        file_path = Path(data_directory) / filename

        df_vol = pd.read_csv(
            file_path,
            usecols=["Date", "Adj Close", "Volume"],
            parse_dates=["Date"]
        )
        df_vol.set_index("Date", inplace=True)
        df_vol.rename(
            columns={"Adj Close": f"{stock_id}_price", "Volume": f"{stock_id}_vol"},
            inplace=True
        )
        dataframes_vol.append(df_vol)

    combined_df_vol = pd.concat(dataframes_vol, axis=1)

    price_columns = [col for col in combined_df_vol.columns if col.endswith('_price')]
    volume_columns = [col for col in combined_df_vol.columns if col.endswith('_vol')]

    combined_df_vol[price_columns] = (
        combined_df_vol[price_columns]
            .ffill()
    )

    combined_df_vol[volume_columns] = (
        combined_df_vol[volume_columns].fillna(0)
    )

    # add a column: how many source tables (stocks) have data on each row
    # (counts a stock as "present" if either its _price or _vol is non-NaN)
    price_vol = combined_df_vol.filter(like='_price')
    present_by_stock = (
        price_vol.notna()
        .groupby(lambda c: c.rsplit("_", 1)[0], axis=1)  # group columns by stock_id
        .any()
    )


    stocks_in_row = present_by_stock.sum(axis=1).astype("int32")
    combined_df_vol[price_columns] = (
        combined_df_vol[price_columns]
            .bfill()
    )

    log_returns_df_vol = np.log(combined_df_vol / (combined_df_vol.shift(1) + 1e-9 ) + 1e-9)
    # drop the first row
    #log_returns_df_vol = log_returns_df_vol.iloc[1:]
    log_returns_df_vol = log_returns_df_vol.dropna()  # Drop the first row with
    log_returns_df_vol = log_returns_df_vol[price_columns + volume_columns]
    # find out the earliest date in the dataframe
    start_date = log_returns_df_vol.index.min().strftime('%Y-%m-%d')
    log_returns_df_vol['stocks_in_row'] = stocks_in_row
    log_returns_df_vol.to_csv(f"stock_data_vol-all-in-one-startdate-{start_date}-stock-nbr.csv")


File: AEP.csv, Rows: 16113
File: BA.csv, Rows: 16113
File: CAT.csv, Rows: 16113


File: CNP.csv, Rows: 16113
File: CVX.csv, Rows: 16113
File: DIS.csv, Rows: 16113
File: DTE.csv, Rows: 16113
File: ED.csv, Rows: 16113
File: GD.csv, Rows: 16113
File: GE.csv, Rows: 16113
File: HON.csv, Rows: 16113
File: HPQ.csv, Rows: 16113
File: IBM.csv, Rows: 16113
File: IP.csv, Rows: 16113
File: JNJ.csv, Rows: 16113
File: KO.csv, Rows: 16113
File: KR.csv, Rows: 16113
File: MMM.csv, Rows: 16113
File: MO.csv, Rows: 16113
File: MRK.csv, Rows: 16113
File: MSI.csv, Rows: 16113
File: PG.csv, Rows: 16113
File: XOM.csv, Rows: 16113
File: LMT.csv, Rows: 16112
File: RTX.csv, Rows: 16050
File: MCD.csv, Rows: 14978


/tmp/ipykernel_6643/4019959411.py:47: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(lambda c: c.rsplit("_", 1)[0], axis=1)  # group columns by stock_id


File: AXP.csv, Rows: 13514
File: BMY.csv, Rows: 13514
File: DD.csv, Rows: 13514
File: DE.csv, Rows: 13514
File: EMR.csv, Rows: 13514
File: ETN.csv, Rows: 13514
File: ETR.csv, Rows: 13514
File: F.csv, Rows: 13514
File: HAL.csv, Rows: 13514
File: LLY.csv, Rows: 13514
File: PCG.csv, Rows: 13514
File: PEP.csv, Rows: 13514
File: PFE.csv, Rows: 13514
File: TXN.csv, Rows: 13514
File: WFC.csv, Rows: 13514
File: WMT.csv, Rows: 13454
File: HUBB.csv, Rows: 13392
File: AIG.csv, Rows: 13368
File: IPG.csv, Rows: 13358
File: K.csv, Rows: 13336
File: AVY.csv, Rows: 13334
File: BAC.csv, Rows: 13334
File: BALL.csv, Rows: 13334
File: BDX.csv, Rows: 13334
File: CLX.csv, Rows: 13334
File: CMI.csv, Rows: 13334
File: CMS.csv, Rows: 13334
File: CPB.csv, Rows: 13334
File: CVS.csv, Rows: 13334
File: ECL.csv, Rows: 13334
File: ES.csv, Rows: 13334
File: EVRG.csv, Rows: 13334
File: GWW.csv, Rows: 13334
File: LNT.csv, Rows: 13334
File: MKC.csv, Rows: 13334
File: MMC.csv, Rows: 13334
File: NEE.csv, Rows: 13334
File:

/tmp/ipykernel_6643/4019959411.py:47: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(lambda c: c.rsplit("_", 1)[0], axis=1)  # group columns by stock_id
/tmp/ipykernel_6643/4019959411.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_returns_df_vol['stocks_in_row'] = stocks_in_row


...

File: O.csv, Rows: 7859
File: MCO.csv, Rows: 7850
File: SJM.csv, Rows: 7850
File: MCK.csv, Rows: 7842
File: COF.csv, Rows: 7838
File: DLTR.csv, Rows: 7763
File: COR.csv, Rows: 7743
File: DRI.csv, Rows: 7719
File: RMD.csv, Rows: 7702
File: WAB.csv, Rows: 7692
File: FCX.csv, Rows: 7677
File: IVZ.csv, Rows: 7643
File: ACGL.csv, Rows: 7630
File: EG.csv, Rows: 7617
File: POOL.csv, Rows: 7609
File: DVA.csv, Rows: 7597
File: HSIC.csv, Rows: 7594
File: EL.csv, Rows: 7584
File: WAT.csv, Rows: 7584
File: NTAP.csv, Rows: 7582
File: HIG.csv, Rows: 7565
File: IRM.csv, Rows: 7533
File: FDS.csv, Rows: 7430
File: STLD.csv, Rows: 7327
File: DGX.csv, Rows: 7311
File: KMX.csv, Rows: 7278
File: TTWO.csv, Rows: 7230
File: VTR.csv, Rows: 7216
File: AMZN.csv, Rows: 7208
File: ARE.csv, Rows: 7200
File: RL.csv, Rows: 7189
File: BXP.csv, Rows: 7185
File: YUM.csv, Rows: 7122
File: CHRW.csv, Rows: 7101
File: FE.csv, Rows: 7084
File: MTD.csv, Rows: 7080
File: PLD.csv, Rows: 7075
File: URI.csv, Rows: 7057
File: AEE

/tmp/ipykernel_6643/4019959411.py:47: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(lambda c: c.rsplit("_", 1)[0], axis=1)  # group columns by stock_id
/tmp/ipykernel_6643/4019959411.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_returns_df_vol['stocks_in_row'] = stocks_in_row


File: NVDA.csv, Rows: 6783
File: BKNG.csv, Rows: 6736
File: GS.csv, Rows: 6713
File: FFIV.csv, Rows: 6691
File: SBAC.csv, Rows: 6683
File: BLK.csv, Rows: 6608
File: AKAM.csv, Rows: 6588
File: UPS.csv, Rows: 6580
File: A.csv, Rows: 6574
File: TDY.csv, Rows: 6571
File: PKG.csv, Rows: 6526
File: EW.csv, Rows: 6486
File: MET.csv, Rows: 6479
File: ON.csv, Rows: 6461
File: ISRG.csv, Rows: 6429
File: CRL.csv, Rows: 6424
File: ILMN.csv, Rows: 6400
File: EQIX.csv, Rows: 6390
File: TPR.csv, Rows: 6351
File: GRMN.csv, Rows: 6307
File: GPN.csv, Rows: 6283
File: ALGN.csv, Rows: 6273
File: WTW.csv, Rows: 6181
File: MDLZ.csv, Rows: 6180
File: AXON.csv, Rows: 6176
File: FIS.csv, Rows: 6175
File: ACN.csv, Rows: 6155
File: ZBH.csv, Rows: 6151
File: BG.csv, Rows: 6145
File: PFG.csv, Rows: 6092
File: ELV.csv, Rows: 6087
File: CNC.csv, Rows: 6056
File: PRU.csv, Rows: 6056
File: NFLX.csv, Rows: 5946
File: NDAQ.csv, Rows: 5920
File: WYNN.csv, Rows: 5838
File: CME.csv, Rows: 5809
File: STX.csv, Rows: 5806


/tmp/ipykernel_6643/4019959411.py:47: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(lambda c: c.rsplit("_", 1)[0], axis=1)  # group columns by stock_id
/tmp/ipykernel_6643/4019959411.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  log_returns_df_vol['stocks_in_row'] = stocks_in_row


KeyboardInterrupt: 

In [15]:
combined_df_vol.tail()

,HPQ_price,HPQ_vol,PG_price,PG_vol,IP_price,IP_vol,DIS_price,DIS_vol,MRK_price,MRK_vol,...,CTEC.L_vol,HWM_price,HWM_vol,LW_price,LW_vol,COWZ_price,COWZ_vol,INVH_price,INVH_vol,tables_in_row
Date,,,,,,,,,,,,,,,,,,,,,
2025-10-20,27.66,7049700.0,151.960007,5601300.0,47.889999,3292900.0,111.959999,8047500.0,86.320000,8611800.0,...,5181799.0,192.520004,1044900.0,65.669998,2489300.0,57.380001,2153200.0,28.600000,4073700.0,696
2025-10-21,28.16,8330400.0,151.619995,6013400.0,47.680000,3301100.0,114.300003,8163200.0,87.620003,9852400.0,...,8403771.0,197.179993,2022600.0,64.650002,2410800.0,57.880001,1698600.0,28.690001,4045700.0,696
2025-10-22,27.66,8976800.0,152.199997,6394400.0,47.950001,2716900.0,113.080002,7569200.0,87.610001,10926300.0,...,11076252.0,191.229996,2251000.0,64.750000,1441400.0,57.660000,1657100.0,28.889999,3504500.0,696
2025-10-23,27.83,11798700.0,152.210007,8205400.0,49.099998,4571500.0,113.029999,6945400.0,87.180000,7687500.0,...,4036832.0,200.100006,2300800.0,65.160004,1448200.0,57.939999,1833500.0,28.910000,2626300.0,696
2025-10-24,27.66,8626235.0,152.490005,9647790.0,48.869999,3862174.0,111.680000,6225963.0,87.489998,5503322.0,...,6243046.0,198.509995,2142300.0,66.379997,1528198.0,57.980000,1874400.0,28.870001,4100900.0,696


In [16]:
len(combined_df_vol.columns)

1393

In [11]:
# to get a big table for all stocks
dataframes_vol = []
for cluster_id in range(len(clusters)-1):
   
    for filename, rows in clusters[cluster_id]:
        print(f"File: {filename}, Rows: {rows}")
        # make one single pandas dataframe contains the data from above stock IDs, the index is the data, the columns are the stock IDs
        stock_id = filename[:-4]  

        file_path = Path(data_directory) / filename
        df_vol = pd.read_csv(file_path, usecols=['Date', 'Adj Close', 'Volume'], parse_dates=['Date'])
        df_vol.set_index('Date', inplace=True)
        df_vol.rename(columns={'Adj Close': f"{stock_id}_price", 'Volume': f"{stock_id}_vol"}, inplace=True)
        dataframes_vol.append(df_vol)

combined_df_vol = pd.concat(dataframes_vol, axis=1)

price_columns = [col for col in combined_df_vol.columns if col.endswith('_price')]
volume_columns = [col for col in combined_df_vol.columns if col.endswith('_vol')]

combined_df_vol[price_columns] = (
    combined_df_vol[price_columns]
        .ffill()
        .bfill()
)

combined_df_vol[volume_columns] = (
    combined_df_vol[volume_columns].fillna(0)
)

log_returns_df_vol = np.log(combined_df_vol / (combined_df_vol.shift(1) + 1e-9 ) + 1e-9)
# drop the first row
#log_returns_df_vol = log_returns_df_vol.iloc[1:]
log_returns_df_vol = log_returns_df_vol.dropna()  # Drop the first row with

price_columns = [col for col in log_returns_df_vol.columns if col.endswith('_price')]
volume_columns = [col for col in log_returns_df_vol.columns if col.endswith('_vol')]
log_returns_df_vol = log_returns_df_vol[price_columns + volume_columns]
# find out the earliest date in the dataframe
start_date = log_returns_df_vol.index.min().strftime('%Y-%m-%d')
log_returns_df_vol.to_csv(f"stock_data_vol-all-in-one-startdate-{start_date}.csv")


File: HPQ.csv, Rows: 16062
File: PG.csv, Rows: 16062
File: IP.csv, Rows: 16062
File: DIS.csv, Rows: 16062
File: MRK.csv, Rows: 16062
File: HON.csv, Rows: 16062
File: CAT.csv, Rows: 16062
File: GE.csv, Rows: 16062
File: IBM.csv, Rows: 16062
File: GD.csv, Rows: 16062
File: JNJ.csv, Rows: 16062
File: ED.csv, Rows: 16062
File: DTE.csv, Rows: 16062
File: XOM.csv, Rows: 16062
File: MMM.csv, Rows: 16062
File: CVX.csv, Rows: 16062
File: BA.csv, Rows: 16062
File: KO.csv, Rows: 16062
File: MSI.csv, Rows: 16062
File: CNP.csv, Rows: 16062
File: MO.csv, Rows: 16062
File: KR.csv, Rows: 16062
File: AEP.csv, Rows: 16062
File: LMT.csv, Rows: 16061
File: RTX.csv, Rows: 15999
File: MCD.csv, Rows: 14927
File: FCIT.L.csv, Rows: 14660
File: SMT.L.csv, Rows: 14660
File: F.csv, Rows: 13463
File: LLY.csv, Rows: 13463
File: HAL.csv, Rows: 13463
File: PCG.csv, Rows: 13463
File: AXP.csv, Rows: 13463
File: EMR.csv, Rows: 13463
File: PFE.csv, Rows: 13463
File: DE.csv, Rows: 13463
File: TXN.csv, Rows: 13463
File: BM

/home/ken/.pyenv/versions/myenv/lib/python3.11/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


Processing cluster 0
File: HPQ.csv, Rows: 16062
File: PG.csv, Rows: 16062
File: IP.csv, Rows: 16062
File: DIS.csv, Rows: 16062
File: MRK.csv, Rows: 16062
File: HON.csv, Rows: 16062
File: CAT.csv, Rows: 16062
File: GE.csv, Rows: 16062
File: IBM.csv, Rows: 16062
File: GD.csv, Rows: 16062
File: JNJ.csv, Rows: 16062
File: ED.csv, Rows: 16062
File: DTE.csv, Rows: 16062
File: XOM.csv, Rows: 16062
File: MMM.csv, Rows: 16062
File: CVX.csv, Rows: 16062
File: BA.csv, Rows: 16062
File: KO.csv, Rows: 16062
File: MSI.csv, Rows: 16062
File: CNP.csv, Rows: 16062
File: MO.csv, Rows: 16062
File: KR.csv, Rows: 16062
File: AEP.csv, Rows: 16062
File: LMT.csv, Rows: 16061
File: RTX.csv, Rows: 15999
File: MCD.csv, Rows: 14927
File: FCIT.L.csv, Rows: 14660
File: SMT.L.csv, Rows: 14660


/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 1
File: F.csv, Rows: 13463
File: LLY.csv, Rows: 13463
File: HAL.csv, Rows: 13463
File: PCG.csv, Rows: 13463
File: AXP.csv, Rows: 13463
File: EMR.csv, Rows: 13463
File: PFE.csv, Rows: 13463
File: DE.csv, Rows: 13463
File: TXN.csv, Rows: 13463
File: BMY.csv, Rows: 13463
File: ETN.csv, Rows: 13463
File: DD.csv, Rows: 13463
File: ETR.csv, Rows: 13463
File: WFC.csv, Rows: 13463
File: PEP.csv, Rows: 13463
File: WMT.csv, Rows: 13403
File: WBA.csv, Rows: 13345


/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


Processing cluster 2
File: AIG.csv, Rows: 13317
File: ES.csv, Rows: 13283
File: CLX.csv, Rows: 13283
File: CVS.csv, Rows: 13283
File: VMC.csv, Rows: 13283
File: BDX.csv, Rows: 13283
File: EVRG.csv, Rows: 13283
File: MKC.csv, Rows: 13283
File: BAC.csv, Rows: 13283
File: TGT.csv, Rows: 13283
File: K.csv, Rows: 13283
File: RVTY.csv, Rows: 13283
File: CMS.csv, Rows: 13283
File: BALL.csv, Rows: 13283
File: TER.csv, Rows: 13283
File: MMC.csv, Rows: 13283
File: XEL.csv, Rows: 13283
File: NEE.csv, Rows: 13283
File: NI.csv, Rows: 13283
File: ECL.csv, Rows: 13283
File: CMI.csv, Rows: 13283
File: PNW.csv, Rows: 13283
File: CPB.csv, Rows: 13283
File: AVY.csv, Rows: 13283
File: SPGI.csv, Rows: 13283
File: TXT.csv, Rows: 13283
File: LNT.csv, Rows: 13283
File: SNA.csv, Rows: 13283
File: GWW.csv, Rows: 13283
File: ITW.csv, Rows: 13269


...

/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'met

File: IEFA.csv, Rows: 3269
File: IXUS.csv, Rows: 3269
File: IEMG.csv, Rows: 3269
File: ABBV.csv, Rows: 3224
Processing cluster 38
File: NCLH.csv, Rows: 3212
File: CCH.L.csv, Rows: 3158
File: IQV.csv, Rows: 3136
File: VNA.DE.csv, Rows: 3122
File: BNDX.csv, Rows: 3119
File: NWS.csv, Rows: 3108
File: NWSA.csv, Rows: 3108
File: CDW.csv, Rows: 3102
File: QUAL.csv, Rows: 3088
Processing cluster 39
File: WLN.PA.csv, Rows: 2902
File: PAYC.csv, Rows: 2901
File: BME.L.csv, Rows: 2874
File: ANET.csv, Rows: 2865
File: DGRO.csv, Rows: 2861
File: IUSB.csv, Rows: 2861


/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)


File: SYF.csv, Rows: 2827
File: ZAL.DE.csv, Rows: 2812
File: CZR.csv, Rows: 2791
File: CFG.csv, Rows: 2789
File: KEYS.csv, Rows: 2771
File: QRVO.csv, Rows: 2720
Processing cluster 40
File: ETSY.csv, Rows: 2649
File: SHOP.csv, Rows: 2625
File: PYPL.csv, Rows: 2594
File: KHC.csv, Rows: 2594
File: 1COV.DE.csv, Rows: 2558
File: VTEB.csv, Rows: 2558
File: SINCH.ST.csv, Rows: 2528
Processing cluster 41
File: FTV.csv, Rows: 2342
File: TTD.csv, Rows: 2287
File: CTEC.L.csv, Rows: 2272
File: HWM.csv, Rows: 2258
File: LW.csv, Rows: 2251
File: COWZ.csv, Rows: 2222
File: INVH.csv, Rows: 2196


/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:16: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='ffill', inplace=True)
/tmp/ipykernel_10246/2627308016.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df_vol.fillna(method='bfill', inplace=True)
